**Set environment**

In [1]:
suppressWarnings(suppressMessages(source("../config/config_sing.R")))
suppressWarnings(suppressMessages(library("GenomicRanges")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


## Import data

In [2]:
fdiry = file.path(FD_RES, "results", "region")
fname = "region_screened_crispri_flowfish.bed"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, col_names = FALSE)
colnames(dat) = c("Chrom", "Start", "End", "Gene")
dat_crispr_region = dat
print(dim(dat))
dat

Rows: 13 Columns: 4
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): X1, X4
dbl (2): X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 13  4


Chrom,Start,End,Gene
<chr>,<dbl>,<dbl>,<chr>
chr11,4091884,6419310,"HBE1,HBG1,HBG2,HBS1L,MYB"
chr11,33064196,34664117,"CAPRIN1,CAT,LMO2"
chr11,61788524,61897153,"FADS1,FADS2,FADS3,FEN1"
chr12,54300766,54301042,"GATA1,HDAC6"
chr12,110726151,112325737,ERP29
chr19,12887110,12887237,"GATA1,HDAC6"
chr3,128487571,128487937,"GATA1,HDAC6"
chr4,55237590,56068531,NMU
chr5,87611207,89911163,MEF2C


In [3]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_hcrff")
fname = "K562_HCRFF_20genes_TSS_TES_coords.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)
dat_crispr_gene_coord = dat
print(dim(dat))
dat

Rows: 20 Columns: 8
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): Chrom, Strand, Coord, Gene
dbl (4): Start, End, TSS, TES

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 20  8


Chrom,Start,End,Strand,TSS,TES,Coord,Gene
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr11,34051730,34102610,+,34051730,34102610,chr11:34051730-34102610,CAPRIN1
chr11,34438933,34472060,+,34438933,34472060,chr11:34438933-34472060,CAT
chr6,109366513,109382467,-,109382467,109366513,chr6:109366513-109382467,CD164
chr12,112013347,112023220,+,112013347,112023220,chr12:112013347-112023220,ERP29
chr11,61799628,61817003,-,61817003,61799628,chr11:61799628-61817003,FADS1
chr11,61816202,61867354,+,61816202,61867354,chr11:61816202-61867354,FADS2
chr11,61873525,61891545,-,61891545,61873525,chr11:61873525-61891545,FADS3
chr11,61792910,61797238,+,61792910,61797238,chr11:61792910-61797238,FEN1
chrX,48786589,48794310,+,48786589,48794310,chrX:48786589-48794310,GATA1


In [4]:
dat = dat_crispr_region
dat = dat %>% tidyr::separate_rows(Gene)
colnames(dat) = c("Chrom", "Start", "End", "Gene")
dat1 = dat

dat = dat_crispr_gene_coord
dat = dat %>% dplyr::select(Chrom, TSS, Gene)
dat2 = dat

dat = dplyr::left_join(dat1, dat2, by=c("Chrom", "Gene")) %>% 
    dplyr::select(Chrom, Start, End, TSS, Gene) %>%
    na.omit

dat_crispr_gene_region = dat
print(dim(dat))
dat

[1] 20  5


Chrom,Start,End,TSS,Gene
<chr>,<dbl>,<dbl>,<dbl>,<chr>
chr11,4091884,6419310,5269945,HBE1
chr11,4091884,6419310,5249857,HBG1
chr11,4091884,6419310,5254781,HBG2
chr11,33064196,34664117,34051730,CAPRIN1
chr11,33064196,34664117,34438933,CAT
chr11,33064196,34664117,33869878,LMO2
chr11,61788524,61897153,61817003,FADS1
chr11,61788524,61897153,61816202,FADS2
chr11,61788524,61897153,61891545,FADS3


## Generate script column

```
python ./scripts/gen_4c_track.py \
       --fpath_hic ${FD_RES}/source/hic_intact_ENCSR479XDG/ENCFF621AIY.hic \
       --normalization NONE \
       --resolution 2_000 \
       --chrom chrX \
       --x1 48_000_000 \
       --y1 50_000_000 \
       --x2 48_800_000 \
       --y2 48_804_000 \
       --verbose
```

In [5]:
print(FD_PRJ)
print(FD_RES)

[1] "/data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC"
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc"


In [6]:
###
fpath_script  = file.path(FD_PRJ, "notebooks", "42_hic_coverage", "gen_virtual4c_track.py")
fpath_hic     = file.path(FD_RES, "source",  "hic_intact_K562_ENCSR479XDG", "ENCFF621AIY.hic")
fdiry_out     = file.path(FD_RES, "results", "hic_intact_K562_ENCSR479XDG", "coverage_virtual4c_track_crispri_hcrff")
fname_prefix  = paste("virtual4C", "K562", "ENCFF621AIY", sep=".")
normalization = "SCALE"
resolution    = "100"

###
dat = dat_crispr_gene_region
dat = dat %>% 
    dplyr::rowwise() %>% 
    dplyr::mutate(CMD = paste(
        paste("python",          fpath_script),
        paste("--fpath_hic",     fpath_hic),
        paste("--fpath_out",     file.path(
            fdiry_out, 
            paste(fname_prefix, Gene, "TSS", "3K", "bg", sep=".")
        )),
        paste("--normalization", normalization),
        paste("--resolution",    resolution),
        paste("--chrom", Chrom),
        paste("--x1",    Start),
        paste("--y1",    End),
        paste("--x2",    TSS - 3000),
        paste("--y2",    TSS + 3000),
        "--verbose"
))
dat_crispr_gene_region_cmd_3k = dat

###
dat = dat_crispr_gene_region
dat = dat %>% 
    dplyr::rowwise() %>% 
    dplyr::mutate(CMD = paste(
        paste("python",          fpath_script),
        paste("--fpath_hic",     fpath_hic),
        paste("--fpath_out",     file.path(
            fdiry_out, 
            paste(fname_prefix, Gene, "TSS", "1K", "bg", sep=".")
        )),
        paste("--normalization", normalization),
        paste("--resolution",    resolution),
        paste("--chrom", Chrom),
        paste("--x1",    Start),
        paste("--y1",    End),
        paste("--x2",    TSS - 1000),
        paste("--y2",    TSS + 1000),
        "--verbose"
))
dat_crispr_gene_region_cmd_1k = dat

## Save results

In [7]:
fdiry = file.path(".")
fname = "hic_intact_K562_ENCSR479XDG_virtual4c_track_crispri_hcrff_3K.tsv"
fpath = file.path(fdiry, fname)

write_tsv(dat_crispr_gene_region_cmd_3k, fpath)

In [8]:
fdiry = file.path(".")
fname = "hic_intact_K562_ENCSR479XDG_virtual4c_track_crispri_hcrff_1K.tsv"
fpath = file.path(fdiry, fname)

write_tsv(dat_crispr_gene_region_cmd_1k, fpath)